In [ ]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Deep Learning/train.csv')

In [ ]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
df.drop(columns=['id', 'qid1', 'qid2'], inplace=True)

In [ ]:
df.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
df.isna().sum()

,0
question1,1
question2,2
is_duplicate,0


In [ ]:
df.dropna(inplace=True)

In [ ]:
# Convert pandas DataFrame → Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['question1', 'question2', 'is_duplicate', '__index_level_0__'],
    num_rows: 404287
})

In [ ]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'is_duplicate', '__index_level_0__'],
        num_rows: 323429
    })
    test: Dataset({
        features: ['question1', 'question2', 'is_duplicate', '__index_level_0__'],
        num_rows: 80858
    })
})

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "bert-base-uncased" # BERT Model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):                                        # preprocessing data -- tokenization
    return tokenizer(example["question1"], example["question2"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)           # Dynamic Padding

Map:   0%|          | 0/323429 [00:00<?, ? examples/s]

Map:   0%|          | 0/80858 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'is_duplicate', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 323429
    })
    test: Dataset({
        features: ['question1', 'question2', 'is_duplicate', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 80858
    })
})

In [ ]:
tokenized_datasets['train'][0]

{'question1': 'What is the technical support phone number for Linksys routers?',
 'question2': 'What is cisco router technical support phone number?',
 'is_duplicate': 0,
 '__index_level_0__': 319229,
 'input_ids': [101,
  2054,
  2003,
  1996,
  4087,
  2490,
  3042,
  2193,
  2005,
  6971,
  7274,
  2799,
  2869,
  1029,
  102,
  2054,
  2003,
  26408,
  2799,
  2099,
  4087,
  2490,
  3042,
  2193,
  1029,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['question1', 'question2', '__index_level_0__'])
tokenized_datasets = tokenized_datasets.rename_column("is_duplicate", "labels")
tokenized_datasets.set_format("torch")

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 323429
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 80858
    })
})

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=32, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([32]),
 'input_ids': torch.Size([32, 57]),
 'token_type_ids': torch.Size([32, 57]),
 'attention_mask': torch.Size([32, 57])}

In [ ]:
!pip install torch_optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 5.8 MB/s eta 0:00:00


In [ ]:
from accelerate import Accelerator
from transformers import AutoModelForSequenceClassification, get_scheduler
from torch.optim import AdamW

accelerator = Accelerator(mixed_precision="fp16")

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5) # optimizer

train_dl, eval_dl, model, optimizer = accelerator.prepare(     # accelerator to speed up things
    train_dataloader, eval_dataloader, model, optimizer
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(  # learning rate scheduler
    "linear",
    optimizer=optimizer,
    num_warmup_steps=3200,
    num_training_steps=num_training_steps,
)

In [ ]:
num_training_steps

30324

In [ ]:
# Training

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/30324 [00:00<?, ?it/s]

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
# Evaluation

import evaluate

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    accuracy.add_batch(predictions=predictions, references=batch["labels"])
    precision.add_batch(predictions=predictions, references=batch["labels"])
    recall.add_batch(predictions=predictions, references=batch["labels"])
    f1.add_batch(predictions=predictions, references=batch["labels"])

print("Accuracy:", accuracy.compute())
print("Precision:", precision.compute(average="macro"))
print("Recall:", recall.compute(average="macro"))
print("F1:", f1.compute(average="macro"))

Accuracy: {'accuracy': 0.9117217838680155}
Precision: {'precision': 0.9037164852493019}
Recall: {'recall': 0.9083654226089053}
F1: {'f1': 0.9059148270427414}
